In [3]:
## Helper functions

## Good old Euclidean algorithm
def gen_euc_alg(n, m):
    old_r, r = n, m
    old_s, s = 1, 0
    old_t, t = 0, 1

    while r > 0:
        q = old_r // r

        old_r, r = r, old_r - q * r
        old_s, s = s, old_s - q * s
        old_t, t = t, old_t - q * t

    return old_s, old_t, old_r

def is_prime(n):
    i = 2
    while i**2 <= n:
        if n % i == 0:
            return False
        i += 1
    return True


def base_2(n):
    powers = []

    while n > 0: 
        powers.append(n % 2)
        n = n // 2

    return powers
    
def k_power_mod_n(a, k, n):
    _, _, r = gen_euc_alg(a, n)
    if r != 1:
        print(f'(!) gcd({a}, {n}) = {r}')
        return

    # Expand k in base 2
    k_base_2 = base_2(k)

    # Print k as a power of 2s
    k_str = f'{k} = '
    
    for deg in range(len(k_base_2) - 1):
        if k_base_2[deg]: k_str += f'{2**deg} + '
    k_str += f'{2**(len(k_base_2) - 1)}'
    
    # a_powers will store the successive squares of a: a_powers[i] = a^(2^i)
    a_powers = [a]
    outlist = []
    
    # Start the expansion of the result as a product of a^(2^i)
    if k_base_2[0]: 
        ans = a
        outlist.append((a,a))
    else: ans = 1

    # Compute powers of a by successive squaring, use the base 2 expansion of k to evaluate
    for i in range(1, len(k_base_2)):
        a_powers.append(a_powers[i-1]**2 % n)

        if k_base_2[i]: 
            ans = ans * a_powers[i] % n
            outlist.append((f'{a}^{2**i}', a_powers[i]))

    # Return answer as an integer
    return ans

In [4]:
# Create ASCII to text dictionaries
int_to_ASCII = {i: chr(i) for i in range(128)}
ASCII_to_int = {value: key for key, value in int_to_ASCII.items()}

# this is needed for the ephemeral keys
import random

## Takes in a string of text "txt", with public key (g, p, A) 
##     g^a = A (mod p) (a = chosen private key)
## Returns list of encrypted chunks via ElGamel (c1, c2)
## c1 = g^k, c2 = mA^k (k = randomly chosen ephemeral key)
def ElGamel_encrypt(txt, pub_key):
    g, p, A = pub_key
    
    to_encrypt = []

    # Set max word length and split text into chunks
    word_len = len(str(p)) // 3
    chunks = [txt[i : i + word_len] for i in range(0, len(txt), word_len)]

    # Load some ephemeral keys
    eph_keys = []
    
    for i in range(len(chunks)):
        r = 0
        while r != 1:
            # We need gcd(k, p-1) = 1, so keep drawing randomly until you get one which is
            k = random.randint(2, p-1)
            _, _, r = gen_euc_alg(k, p-1)

        eph_keys.append(k)
        
    # Encode txt into ASCII by concatenation
    for chunk in chunks:
        temp_str = ''
        for char in chunk:
            num = str(ASCII_to_int[char])

            # Formatting to deal with leading 0s
            while len(num) < 3: 
                num = '0' + num

            temp_str += num
            
        to_encrypt.append(int(temp_str))

    # This will be the encrypted message
    encrypted = []
    
    for i in range(len(chunks)):
        # c1 = g^k, c2 = mA^k, take everything mod p
        c1 = k_power_mod_n(a=g, k=eph_keys[i], n=p)
        c2 = (to_encrypt[i] * k_power_mod_n(a=A, k=eph_keys[i], n=p)) % p 
        
        encrypted.append((c1, c2))
        
    return encrypted

## Expects encrypted to be a list of entries (c1, c2) encoded by ElGamel
## p = prime number, must provide the private key a to decode
def ElGamel_decrypt(encrypted:list, p, priv_key):
    decrypted = []
    
    for i in range(len(encrypted)):
        c1, c2 = encrypted[i]
        
        # First find the inverse of c1 (mod p)
        inv, _, r = gen_euc_alg(c1, p)

        # Decrypt chunk by chunk via m = c1^-a * c2
        decrypted.append(k_power_mod_n(a=inv, k=priv_key, n=p) * c2 % p) 
    
    to_str = []
    
    for term in decrypted:
        word = str(term)

        # Deal with leading 0s
        while len(word) % 3 != 0:
            word = '0' + word

        # Split into 3-bit ASCII codes
        chunks = [word[i: i+3] for i in range(0, len(word), 3)]
        to_str.append(chunks)
    
    out_str = ''

    # Create the string of text to return
    for word in to_str:
        for entry in word:
            out_str += int_to_ASCII[int(entry) % 128]

    return out_str

In [5]:
#### ELGamel key creator

## Pick a large prime p:
# p = 567891011121314151617 
# p = 1044444166666668888888889999999999
p = 17389

## Pick g so that gcd(g, p-1) = 1
## g should have large order mod p-1 (this is something to check, but is difficult)
# g = 908139481948935983913413341
g = 9704

d, _, r = gen_euc_alg(g, p)

if r != 1:
    print('g must be coprime to p-1')

## Pick private key a so that g^a seems "randomized" mod p
## This a should be chosen so that the discrete log problem g^a = A (mod p) is difficult to solve
#a = 193849819439839841
a = 1159

## Evaluatae A
A = k_power_mod_n(a=g, k=a, n=p)

print('ElGamel encryption:\n')
print('Public Key: (g, p, A)')
print(f'p = {p} ({len(str(p))} digits)')
print(f'g = {g}')
print(f'A = {A}')
print()

print('Private key: a')
print(f'a = {a}')
print(f'g^a = A (mod p)')
print()

ElGamel encryption:

Public Key: (g, p, A)
p = 17389 (5 digits)
g = 9704
A = 13896

Private key: a
a = 1159
g^a = A (mod p)



In [6]:
# Check that our encryption works!
wrong_key = random.randint(1, p-1)

in_txt = 'beans and legumes'

enc = ElGamel_encrypt(in_txt, pub_key=(g,p,A))

out_txt = ElGamel_decrypt(encrypted=enc, p=p, priv_key=a)
out_txt_wrong = ElGamel_decrypt(encrypted=enc, p=p, priv_key=1159)

print('Using ElGamel encryption:\nPublic key (g, p, A) =', (g,p,A), '\nPrivate key a =')
print()

print('Initial message:', in_txt)
print()

print(f'Encoded message: {enc}')
print()

print('Unencrypted:', out_txt)
print()

if in_txt == out_txt:
    print('Success!')

print()
print(f'Attempted decryption with the wrong pivate key ({wrong_key}):', out_txt_wrong)

Using ElGamel encryption:
Public key (g, p, A) = (9704, 17389, 13896) 
Private key a =

Initial message: beans and legumes

Encoded message: [(8206, 13256), (15688, 3724), (7705, 12307), (10255, 10747), (9216, 15828), (14935, 17253), (14044, 16408), (15885, 9917), (15268, 17372), (11451, 6928), (11491, 14277), (1425, 5208), (13835, 7959), (6087, 12815), (13640, 15287), (16180, 14056), (11086, 2747)]

Unencrypted: beans and legumes

Success!

Attempted decryption with the wrong pivate key (14076): beans and legumes


In [7]:
# Try it with this prime (it might crash I don't know)
# p = 12345678900000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001

In [18]:
print(c_1, c_2, mm)

13429939 30907909 12345
